# LangChain: Models, Prompts and Output Parsers

## Outline

 * Direct API calls to GroqAPI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [44]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

def get_completion(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1 # 1 seems to give the best results
    )
    return response.choices[0].message.content


#### The code below used to select the model version at the time of code execution. Nonetheless, because I am using the GroqAPI, I have much more flexibility in choosing the LLM to promt, and the one used in these notebooks is Meta's LLaMA3 70b model with a context window of 8,192 tokens.

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : GroqAI

Let's start with a direct API calls to Groq.

In [29]:
get_completion("What is 1+1?")


"That's an easy one! 1 + 1 = 2."

In [30]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [32]:
style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [64]:
response = get_completion("How can I use Langchain to prompt the GroqAPI?", model="llama3-70b-8192")
print(response)

Langchain is an AI model that can generate natural language prompts, and GroqAPI is a search engine that allows you to query large datasets using natural language queries. Combining the two can be a powerful way to explore and analyze large datasets.

To use Langchain to prompt the GroqAPI, you can follow these general steps:

1. **Generate a prompt with Langchain**: Use Langchain's API or a Langchain-powered interface (e.g., a chatbot or a web app) to generate a natural language prompt. For example, you can ask Langchain to generate a prompt related to a specific topic, like "What are the top 10 most popular books about artificial intelligence?"
2. **Refine the prompt (optional)**: You can refine the generated prompt by modifying it to better suit your needs. This might involve adding or removing keywords, changing the tone, or adjusting the specificity of the question.
3. **Send the prompt to the GroqAPI**: Take the refined prompt and send it to the GroqAPI as a query. You can do thi

In [63]:
response = get_completion("How can I use Langchain to prompt the GroqAPI?", model="mixtral-8x7b-32768")
print(response)

I'm sorry for any confusion, but it seems there may be a misunderstanding. Langchain is a hypothetical concept of a blockchain-based platform for language processing, while GroqAPI is a specific API for querying data on the Groq database system.

To use the GroqAPI, you should follow the documentation provided by Groq. Generally, you will need to:

1. Send a query to the GroqAPI in the format specified by Groq.
2. Handle the response, which will be in a format such as JSON or Protocol Buffers.

Langchain, as a language processing platform, would not directly interact with an API like GroqAPI. Instead, Langchain might use data obtained via an API like GroqAPI for language processing tasks, such as natural language understanding or machine translation.

If you want to use Langchain to process data from GroqAPI, you would first query the GroqAPI, obtain the desired data, then use Langchain to analyze the data and generate insights. However, as Langchain does not currently exist and is pur

## Chat API : LangChain

Let's try how we can do the same using LangChain. To use GroqAPi, we must install 'langchain-groq' instead of 'langchain', see [Official Docs](https://python.langchain.com/v0.2/docs/integrations/chat/groq/) for refrence

In [65]:
#!pip install --upgrade langchain-groq

### Model

In [72]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.0, model_name="llama3-70b-8192")

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."}).content

'Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Conversational AI**: In conversational AI, low-latency LLMs enable more natural and human-like interactions. Fast response times are critical in chatbots, virtual assistants, and voice assistants to maintain user engagement and provide a seamless experience.\n2. **Real-time Language Translation**: Low-latency LLMs are vital in real-time language translation applications, such as simultaneous interpretation, live subtitles, and multilingual customer support. Fast processing ensures that translations are provided quickly, enabling effective communication across languages.\n3. **Sentiment Analysis and Feedback**: In applications like customer feedback analysis, low-latency LLMs help process and analyze user feedback in real-time, enabling businesses to respond promptly to customer concerns

In [70]:
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x72fa96e1f0a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x72fa96e1d690>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [71]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
prompt_